In [ ]:
import pandas as pd
df=pd.read_csv("/Users/anand/Downloads/WA_Fn-UseC_-HR-Employee-Attrition.csv")
df

In [ ]:
#check the null values #no null values
df.isnull().sum()


In [ ]:
#checking datatype
df.dtypes

In [ ]:
#dropping column employee number as it is not needed for any analysis
df.drop(columns=["EmployeeNumber"],inplace=True)

In [ ]:
df["Over18"].unique()
#this columnn has only one value which is Y ,so dropping

In [ ]:
df.drop(columns=["Over18"],inplace=True)

In [ ]:
df["EmployeeCount"].unique()



In [ ]:
df["StandardHours"].unique()

In [ ]:
#dropping EmployeeCount column
df.drop(columns=["EmployeeCount"],inplace=True)

In [ ]:
df.drop(columns=["StandardHours"],inplace=True)

In [ ]:
#count of attrition
import seaborn as sn
import matplotlib.pyplot as plt
sn.countplot(x="Attrition",data=df)
plt.show()
#Imbalanced dataset.

In [ ]:
#Age of employees who is leaving and who are staying
sn.barplot(x="Attrition",y="Age",data=df)
plt.show()
#below graph shows that people who are staying has age more than 35

In [ ]:
#Count of employees based on buisness travel
sn.countplot(x="BusinessTravel",data=df)
plt.show()
#most of the employee travel rarely

In [ ]:
#Count of each category of buisness travel where attrition is true
sn.countplot(x="BusinessTravel",data=df,hue="Attrition")
plt.show()
# proportion of attrition is higher where travel is frequent

In [ ]:
#which department attrintion is high
sn.countplot(x="Department",data=df,hue="Attrition")
plt.show()
#sales 1:4 ,R&D 1:9 

In [ ]:
#when distance from home is above 15 kms whats the attrition
sn.countplot(x="Attrition",data=df.loc[df["DistanceFromHome"]>15])
plt.show()

In [ ]:
#attrition in each job role
plt.figure(figsize=(22,6))
sn.countplot(x="JobRole",data=df,hue="Attrition")
plt.show()


In [ ]:
#attrition ratio based on gender
sn.countplot(x="Gender",data=df,hue="Attrition")
plt.show()

In [ ]:
#attrition ratio based on job satisfaction
sn.countplot(x="JobSatisfaction",data=df,hue="Attrition")
plt.show()
# as job satisfaction increases proportion of attrition decreases

In [ ]:
#attrition ratio in MaritalStatus
sn.countplot(x="MaritalStatus",data=df,hue="Attrition")
plt.show()
#in singles ratio is higher

In [ ]:
#attrition based on overtime
sn.countplot(x="OverTime",data=df,hue="Attrition")
plt.show()
#more overtime more attrition

In [ ]:
sn.countplot(x="YearsWithCurrManager",data=df.loc[df["Attrition"]=="Yes"])
plt.show()
#if years working wth current manager is 0 then attrition was very high .Also after 2 years and at 7 years of working ,more ppl left
#which means in  new project they moved 

In [ ]:
#work life balance with attrition
sn.countplot(x="WorkLifeBalance",data=df.loc[df["Attrition"]=="Yes"])
plt.show()

In [ ]:
# average working years of the employees
sn.barplot(x="Attrition",y="YearsAtCompany",data=df)
plt.show()


In [ ]:
#attritionand years since last promotion
df.loc[df["Attrition"]=="Yes"].groupby(["YearsSinceLastPromotion"]).agg({"Attrition":"count"})

#promotion is not the factor why people left the company 

In [ ]:
#performnce rating and attrition
sn.countplot(x="PerformanceRating",data=df.loc[df["Attrition"]=="Yes"])
plt.show()
# people who left were who got rating 3

In [ ]:
#correlation matrix
df.corr()

In [ ]:
#SKEWNESS
print(df.skew())
print("Total count of numeric features: ",len(df.skew()))
print("count of features which are significantly skewed: ",len(df.skew().loc[abs(df.skew())>0.5]))

In [ ]:
#seprate the input and output 
X=df.drop(columns=["Attrition"])
y=df[["Attrition"]]

In [ ]:
#ELEIMINATING SKEWNWSS
import numpy as np
for index in X.skew().index:
    if X.skew().loc[index]>0.5:
        X[index]=np.log1p(X[index])
    if X.skew().loc[index]<-0.5:
        X[index]=np.square(X[index])

In [ ]:
#skewness
print(X.skew())
print("Total count of numeric features: ",len(X.skew()))
print("count of features which are significantly skewed: ",len(X.skew().loc[abs(X.skew())>0.5]))
#skewness changed from 14 to 3

In [ ]:
#the number of columns of object datatype
print(X.dtypes.loc[X.dtypes=="object"])

In [ ]:
#we need to convert above columns to 1 and 0 using pd.get_dummies
X=pd.get_dummies(X,drop_first=True)

In [ ]:

X

In [ ]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
sc.fit(X)
X_new=sc.transform(X)
X_new=pd.DataFrame(X_new,columns=X.columns)

In [ ]:
#labelencoder to convert target class into integers
from sklearn.preprocessing import LabelEncoder
ls=LabelEncoder()
ls.fit(y)
y=ls.transform(y)
y

In [ ]:
#imbalanced dataset so we will focus on auc-roc score
#!pip install imblearn


from imblearn.over_sampling import SMOTE
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
def max_aucroc_score(clf,X,y):
    max_aucroc_score=0
    for r_state in range(42,100):
        x_train, x_test, y_train, y_test = train_test_split(X, y,random_state = r_state,test_size=0.20,stratify=y)
        x_train, y_train = SMOTE().fit_sample(x_train, y_train)
        clf.fit(x_train,y_train)
        y_pred = clf.predict(x_test)
        aucroc_scr=roc_auc_score(y_test,y_pred)
        print("auc roc score corresponding to ",r_state," is ",aucroc_scr)
        if aucroc_scr>max_aucroc_score:
            max_aucroc_score=aucroc_scr
            final_r_state=r_state
    print("max auc roc score corresponding to ",final_r_state," is ",max_aucroc_score)
    return final_r_state

In [ ]:
#Lets use logistic regression and check
from sklearn.linear_model import LogisticRegression
import warnings
warnings.filterwarnings("ignore")
lg_clf=LogisticRegression()
max_aucroc_score(lg_clf,X_new,y)

In [ ]:
#lets use cross_val_score
from sklearn.model_selection import cross_val_score
print("Mean roc auc score for logistic classifier: ",cross_val_score(lg_clf,X_new,y,cv=5,scoring="roc_auc").mean())
print("standard deviation in roc auc score for logistic classifier: ",cross_val_score(lg_clf,X_new,y,cv=5,scoring="roc_auc").std())
print(cross_val_score(lg_clf,X_new,y,cv=5,scoring="roc_auc"))

In [ ]:
#lets chcek decision tree
from sklearn.tree import DecisionTreeClassifier
dc=DecisionTreeClassifier()
max_aucroc_score(dc,X,y)

In [ ]:
print("Mean auc roc score for decision tree classifier: ",cross_val_score(dc,X,y,cv=5,scoring="roc_auc").mean())
print("standard deviation in auc roc score for decision tree classifier: ",cross_val_score(dc,X,y,cv=5,scoring="roc_auc").std())
print(cross_val_score(dc,X,y,cv=5,scoring="roc_auc"))

In [ ]:
#random forest
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
parameters={"n_estimators":[10,100,500]}
rf_clf=RandomForestClassifier()
clf = GridSearchCV(rf_clf, parameters, cv=5,scoring="roc_auc")
clf.fit(X,y)
clf.best_params_

In [ ]:
rf_clf=RandomForestClassifier(n_estimators=500)
max_aucroc_score(rf_clf,X,y)

In [ ]:
#cross_val_score
print("Mean auc roc score for random forest classifier: ",cross_val_score(rf_clf,X,y,cv=5,scoring="roc_auc").mean())
print("standard deviation in auc roc score for random forest classifier: ",cross_val_score(rf_clf,X,y,cv=5,scoring="roc_auc").std())
print(cross_val_score(rf_clf,X,y,cv=5,scoring="roc_auc"))

In [ ]:
# KNN
#For KNN we need to know the best value of k using grid search
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
import warnings
warnings.filterwarnings("ignore")
kc=KNeighborsClassifier()
neighbors={"n_neighbors":range(1,30)}
clf = GridSearchCV(kc, neighbors, cv=5,scoring="roc_auc")
clf.fit(X_new,y)
clf.best_params_

In [ ]:
kc=KNeighborsClassifier(n_neighbors=28)
max_aucroc_score(kc,X_new,y)

In [ ]:
#SVC
from sklearn.svm import SVC
svc=SVC()
parameters={"kernel":["linear", "poly", "rbf"],"C":[0.001,0.01,0.1,1,10]}
clf = GridSearchCV(svc, parameters, cv=5,scoring="roc_auc")
clf.fit(X_new,y)
clf.best_params_

In [ ]:
svc=SVC(kernel="linear",C=0.1)
max_aucroc_score(svc,X_new,y)

In [ ]:
# Gradient boosting classifier
from sklearn.ensemble import GradientBoostingClassifier
parameters={"learning_rate":[0.001,0.01,0.1,1],"n_estimators":[10,100,500,1000]}
gb_clf=GradientBoostingClassifier()
clf = GridSearchCV(gb_clf, parameters, cv=5,scoring="roc_auc")
clf.fit(X,y)
clf.best_params_

In [ ]:
gb_clf=GradientBoostingClassifier(learning_rate=0.1,n_estimators=500)
max_aucroc_score(gb_clf,X,y)

In [ ]:
#final modelsvc 
x_train, x_test, y_train, y_test = train_test_split(X_new, y,random_state = 70,test_size=0.20,stratify=y)
x_train, y_train = SMOTE().fit_sample(x_train, y_train)
svc=SVC(kernel="linear",C=0.1)
svc.fit(x_train,y_train)
y_pred=svc.predict(x_test)

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
print("Confusion matrix \n",confusion_matrix(y_test,y_pred))
print("f1 score is : ",f1_score(y_test,y_pred))
print("classification report \n",classification_report(y_test,y_pred))
print("AUC ROC Score: ",roc_auc_score(y_test,y_pred))

In [102]:
import joblib
joblib.dump(svc,"HR_Analytics.pkl")

['HR_Analytics.pkl']